In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import re



In [27]:
#landing page

landing_page = "https://www.amazon.com/s?i=computers-intl-ship&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011%2Cp_n_deal_type%3A23566065011&dc&ds=v1%3A0skNnwRMDjZUviLW2wbTrH0XVm9pFV3leK3HXNOFM6Q&qid=1708154445&rnid=23566063011&ref=sr_nr_p_n_deal_type_1"


#defining the HEADERS
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [28]:
#get the http request

request = requests.get(url= landing_page, headers= HEADERS)

In [19]:
#vieew all the atrributes

request.__attrs__

['_content',
 'status_code',
 'headers',
 'url',
 'history',
 'encoding',
 'reason',
 'cookies',
 'elapsed',
 'request']

In [29]:
#view the status code

request.status_code

200

In [30]:
#creating soup

soup = BeautifulSoup(markup= request.content, 
                     features= 'html.parser')

In [31]:
ProductList = soup.find_all(name= 'div', 
                            attrs= {"class": "a-section a-spacing-small puis-padding-left-small puis-padding-right-small"})

In [32]:
#first product
test = ProductList[0]

test

<div class="a-section a-spacing-small puis-padding-left-small puis-padding-right-small"><div class="a-section a-spacing-none a-text-center"><div class="a-section s-color-swatch-container s-quick-view-text-align-start"><div data-csa-c-content-id="more-colors-message" data-csa-c-interaction-events="click" data-csa-c-swatch-more-url="/sspa/click?ie=UTF8&amp;spc=MTo0NzcyMzAzMTI0NjU2MDQ3OjE3MDgxNjExOTA6c3BfYXRmX2Jyb3dzZTozMDAwMzc4OTI5MzYzMDI6OjA6Og&amp;url=%2FTablets-Android-Expand-Quad-Core-Camera%2Fdp%2FB0BFWWZ3PS%2Fref%3Dacm_sr_dp_sspa%3Fdib%3DeyJ2IjoiMSJ9.wxL_4aeDzhJLlBo6Kx-on7rOeA-9ymRX1N7QZZqmtCykC70M9_c3E3FaUrTMNVWgHtmKvcarThxR3jFO11WcaCHcZi_sv3E7skoGz4Etd2OIgrYPIRcQMeGWhPWlIHjcU9dK5R4dNhEc14xc_BhVrv-pdbOQtm00U5MC9cdiZYLMvYdfOQqZ0WBWwNvMdQkJMStMqmnGK5SoAdOOwAIMszl-V2VM8Oh610Few13yd2DYprP8uj4l2UlWY6zLwaZ-QnkVkyIEA1Bp42IzavhTpklt5cgZ1ahBHhtYHU4Xt7U.oWm8DSwWQUnBGXGu9Hpe-dKlmjOb29tnkiHc-ZR-zaU%26dib_tag%3Dse%26qid%3D1708161190%26refinements%3Dp_n_deal_type%253A23566065011%26rnid%3D235660

In [37]:
#finding the product title
title = test.find(name= 'h2', attrs= {'class': 'a-size-mini a-spacing-none a-color-base s-line-clamp-4'})\
     .find(name  = 'a', attrs = {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})\
     .find(name = 'span', attrs = {'class': 'a-size-base-plus a-color-base a-text-normal'}).text

In [84]:
#Extracting price after discount

PriceAfterDiscount = test.find(name= 'div',attrs= {'data-cy': "price-recipe"})\
                         .find(name = 'div', attrs = {'class': "a-row a-size-base a-color-base"})\
                         .find(name = 'div', attrs = {'class': 'a-row'})\
                         .find(name = 'span', attrs = {'class': 'a-price'}).text.split('$')[1]

'59.99'

In [127]:
#Etracting price before discount

PriceBeforeDiscount = test.find(name= 'div',attrs= {'data-cy': "price-recipe"})\
                          .find(name = 'div', attrs = {'class': "a-row a-size-base a-color-base"})\
                          .find(name = 'div', attrs = {'class': 'a-row'})\
                          .find(name = 'a', attrs = {'class': 'a-link-normal s-no-hover s-underline-text s-underline-link-text s-link-style a-text-normal'})\
                          .find(name = 'div', attrs = {'class': 'a-section aok-inline-block'})\
                          .find(name = 'span', attrs = {'class': 'a-price a-text-price'}).text.split('$')[1]
PriceBeforeDiscount

'69.99'

In [104]:
#Etracting product rating out of five
ProductRating = test.find(name = 'div', attrs= {'class': 'a-section a-spacing-none a-spacing-top-micro'})\
                    .text.split()[0]

In [148]:
#Combing the results for the whole page

df = pd.DataFrame(columns=['Title', 'PriceAfterDiscount','PriceBeforeDiscount', 'ProductRating'])

for idx, product in enumerate(ProductList):

    #Extract title

    Title = product.find(name= 'h2', attrs= {'class': 'a-size-mini a-spacing-none a-color-base s-line-clamp-4'})\
                   .find(name  = 'a', attrs = {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})\
                   .find(name = 'span', attrs = {'class': 'a-size-base-plus a-color-base a-text-normal'}).text
    
    #Extracting price after discount

    PriceAfterDiscount = product.find(name= 'div',attrs= {'data-cy': "price-recipe"})\
                                .find(name = 'div', attrs = {'class': "a-row a-size-base a-color-base"})\
                                .find(name = 'div', attrs = {'class': 'a-row'})\
                                .find(name = 'span', attrs = {'class': 'a-price'}).text.split('$')[1]
    
    #Etracting price before discount
    PriceBeforeDiscount = product.find(name= 'div',attrs= {'data-cy': "price-recipe"})\
                          .find(name = 'div', attrs = {'class': "a-row a-size-base a-color-base"})\
                          .find(name = 'div', attrs = {'class': 'a-row'})\
                          .find(name = 'a', attrs = {'class': 'a-link-normal s-no-hover s-underline-text s-underline-link-text s-link-style a-text-normal'})\
                          .text.split('$')[-1]

    
    #Etracting product rating out of five
    ProductRating = product.find(name = 'div', attrs= {'class': 'a-section a-spacing-none a-spacing-top-micro'})\
                           .text.split()[0]
    

    df.loc[idx] = [Title, PriceAfterDiscount, PriceBeforeDiscount ,ProductRating]
    



    


In [149]:
df.head()

,Title,PriceAfterDiscount,PriceBeforeDiscount,ProductRating
0,"2024 Newest 10 inch Tablets, Android Tablets, ...",59.99,69.99,3.9
1,"Tablet, Android 13 Tablet, 64GB ROM 6GB RAM (1...",59.99,79.99,5.0
2,Android 12 Tablet 10 Inch Tablet with Keyboard...,89.99,99.99,4.2
3,"HP Notebook Laptop, 15.6"" HD Touchscreen, Inte...",472.80,699.00,4.3
4,"SAMSUNG Galaxy Tab S6 Lite 10.4"" 128GB Android...",299.99,429.99,4.7


## Adding Pagination

In [145]:
PagesScrapped = 1

df = pd.DataFrame(columns=['Title', 'PriceAfterDiscount','PriceBeforeDiscount', 'ProductRating'])

landing_page = 'https://www.amazon.com/s?i=computers-intl-ship&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011%2Cp_n_deal_type%3A23566065011&dc&ds=v1%3A0skNnwRMDjZUviLW2wbTrH0XVm9pFV3leK3HXNOFM6Q&qid=1708154445&rnid=23566063011&ref=sr_nr_p_n_deal_type_1'

url = landing_page

listofDFs = []

while PagesScrapped < 91:
    #Making a request for each page
    response = requests.get(url= url, headers= HEADERS)

    #create a soup for each page

    soup = BeautifulSoup(markup= response.content, features= 'html.parser')

    

    






